# Data Pipeline: File Ingest Workflows

This notebook demonstrates how to upload files to the Ocean Data Platform and ingest them into queryable tabular datasets.

**What you'll learn:**
- Create a personal dataset (via web UI)
- Upload files to an ODP dataset
- Manage file metadata (name, MIME type, geometry)
- Ingest files into tabular format
- Handle different ingest modes (append, truncate, drop)

**Prerequisites:**
- Running in ODP Workspace (auto-authenticated)
- An ODP account with access to My Data

## Before You Begin: Create Your Dataset

The ODP Python SDK requires a dataset UUID to work with. Dataset creation is done via the web interface:

1. Go to [app.hubocean.earth](https://app.hubocean.earth)
2. Log in with your credentials
3. Navigate to **My Data** section
4. Click **Create Dataset** (or similar)
5. Give it a name (e.g., "Pipeline Tutorial Data")
6. Copy the **UUID** shown in the dataset details

You'll enter this UUID when running the notebook below.

> **Note:** Programmatic dataset creation (`client.create_dataset()`) and listing personal datasets (`client.list_my_datasets()`) are not currently available in the SDK. See `proposals/my_data_api.md` for a feature proposal.

## 1. Setup

In [ ]:
from odp.client import Client
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta

# Initialize ODP client
client = Client()
print("Client initialized")

## 2. Connect to Your Dataset

Enter your dataset ID where you have editor permissions.

In [ ]:
# Enter your dataset UUID from My Data
DATASET_ID = input("Enter your dataset UUID (from app.hubocean.earth → My Data): ").strip()

if not DATASET_ID:
    raise ValueError("Dataset UUID is required. Create one at app.hubocean.earth → My Data")

dataset = client.dataset(DATASET_ID)
print(f"Connected to dataset: {DATASET_ID}")

# Check current state
files = dataset.files.list()
schema = dataset.table.schema()

print(f"\nCurrent state:")
print(f"  Files: {len(files)}")
print(f"  Table: {'exists' if schema else 'none'}")

## 3. Generate Sample Oceanographic Data

We'll create realistic Norwegian Sea monitoring station data.

In [ ]:
# Norwegian Sea monitoring stations
np.random.seed(42)  # Reproducible

stations = [
    {"id": "OSLO-01", "name": "Oslofjord Outer", "lat": 59.82, "lon": 10.52},
    {"id": "BERG-01", "name": "Bergen Harbor", "lat": 60.39, "lon": 5.32},
    {"id": "STAV-01", "name": "Stavanger Approach", "lat": 58.97, "lon": 5.73},
    {"id": "TRND-01", "name": "Trondheim Fjord", "lat": 63.43, "lon": 10.40},
    {"id": "TRMS-01", "name": "Tromsø Sound", "lat": 69.65, "lon": 18.96},
    {"id": "LOFT-01", "name": "Lofoten Basin", "lat": 68.50, "lon": 14.00},
    {"id": "NRTH-01", "name": "North Cape", "lat": 71.17, "lon": 25.78},
    {"id": "SVAL-01", "name": "Svalbard South", "lat": 76.50, "lon": 16.00},
]

# Generate time series (30 days of readings)
start_date = datetime(2024, 6, 1)
records = []

for day in range(30):
    timestamp = start_date + timedelta(days=day)
    
    for station in stations:
        # Temperature varies by latitude and season
        base_temp = 15 - (station["lat"] - 58) * 0.3
        temp = base_temp + np.random.normal(0, 1.5)
        
        # Salinity relatively stable
        salinity = 34.5 + np.random.normal(0, 0.5)
        
        # Depth varies by station
        depth = np.random.choice([5, 10, 25, 50, 100])
        
        records.append({
            "station_id": station["id"],
            "station_name": station["name"],
            "latitude": station["lat"],
            "longitude": station["lon"],
            "timestamp": timestamp.isoformat(),
            "depth_m": depth,
            "temperature_c": round(temp, 2),
            "salinity_psu": round(salinity, 2),
            "dissolved_oxygen_ml_l": round(6.5 + np.random.normal(0, 0.8), 2),
            "chlorophyll_ug_l": round(max(0.1, 2.0 + np.random.normal(0, 1.0)), 2)
        })

df_stations = pd.DataFrame(records)
print(f"Generated {len(df_stations)} observations from {len(stations)} stations over 30 days")
print(f"\nColumns: {list(df_stations.columns)}")
df_stations.head(10)

In [ ]:
# Summary statistics
print("Data Summary:")
print(f"  Date range: {df_stations['timestamp'].min()} to {df_stations['timestamp'].max()}")
print(f"  Latitude range: {df_stations['latitude'].min():.2f}°N to {df_stations['latitude'].max():.2f}°N")
print(f"  Temperature range: {df_stations['temperature_c'].min():.1f}°C to {df_stations['temperature_c'].max():.1f}°C")
print(f"\nObservations per station:")
print(df_stations['station_name'].value_counts())

## 4. Upload Data as CSV

In [ ]:
# Convert DataFrame to CSV bytes
csv_buffer = io.BytesIO()
df_stations.to_csv(csv_buffer, index=False)
csv_bytes = csv_buffer.getvalue()

print(f"CSV size: {len(csv_bytes):,} bytes")
print(f"\nFirst 500 chars:\n{csv_bytes[:500].decode()}")

In [ ]:
# Upload to ODP
filename = "norwegian_sea_monitoring_2024.csv"
file_id = dataset.files.upload(filename, csv_bytes)

print(f"Uploaded!")
print(f"  Filename: {filename}")
print(f"  File ID: {file_id}")

## 5. Update File Metadata

Add geographic extent and proper MIME type.

In [ ]:
# Calculate bounding box from data
min_lon = df_stations['longitude'].min()
max_lon = df_stations['longitude'].max()
min_lat = df_stations['latitude'].min()
max_lat = df_stations['latitude'].max()

# Create WKT polygon (with small buffer)
bbox_wkt = f"POLYGON(({min_lon-1} {min_lat-1}, {max_lon+1} {min_lat-1}, {max_lon+1} {max_lat+1}, {min_lon-1} {max_lat+1}, {min_lon-1} {min_lat-1}))"

print(f"Bounding box: {min_lat:.2f}°N to {max_lat:.2f}°N, {min_lon:.2f}°E to {max_lon:.2f}°E")
print(f"WKT: {bbox_wkt}")

In [ ]:
# Update metadata
dataset.files.update_meta(file_id, {
    "name": filename,
    "mime-type": "text/csv",
    "geometry": bbox_wkt
})

print("Metadata updated!")

# Verify
files = dataset.files.list()
for f in files:
    if f.get('id') == file_id:
        print(f"\nFile details:")
        print(f"  Name: {f.get('name')}")
        print(f"  Size: {f.get('size'):,} bytes")
        print(f"  MIME: {f.get('mime-type')}")
        print(f"  Geometry: {f.get('geometry', 'N/A')[:60]}...")

## 6. Ingest into Table

Convert the CSV file into a queryable table.

**Ingest modes:**
- `drop`: Recreate table from scratch (use for first ingest)
- `truncate`: Clear data but keep schema
- `append`: Add rows to existing table

In [ ]:
# Ingest file into table
print("Ingesting CSV into table...")

dataset.files.ingest(file_id, opt="drop")  # Creates fresh table

print("Ingest complete!")

In [ ]:
# Verify table creation
schema = dataset.table.schema()

if schema:
    print(f"Table schema ({len(schema)} columns):")
    for field in schema:
        print(f"  {field.name}: {field.type}")
    
    stats = dataset.table.stats()
    if stats:
        print(f"\nTable statistics:")
        print(f"  Rows: {stats.num_rows:,}")
        print(f"  Size: {stats.size:,} bytes")
else:
    print("Table not created - check ingest errors above")

## 7. Query the Ingested Data

In [ ]:
# Query all data
df_query = dataset.table.select().all().dataframe()

print(f"Queried {len(df_query)} rows")
df_query.head(10)

In [ ]:
# Query with filter
cold_water = dataset.table.select(
    "temperature_c < $threshold",
    vars={"threshold": 5.0}
).all().dataframe()

print(f"Cold water observations (<5°C): {len(cold_water)}")
if len(cold_water) > 0:
    print(f"\nStations with cold water:")
    print(cold_water['station_name'].value_counts())

In [ ]:
# Aggregation
station_avg = dataset.table.aggregate(
    group_by="station_name",
    aggr={
        "temperature_c": "mean",
        "salinity_psu": "mean",
        "*": "count"
    }
)

print("Average conditions by station:")
station_avg.sort_values('mean_temperature_c', ascending=False)

## 8. Append Additional Data

Add more observations using `append` mode.

In [ ]:
# Generate July data (next 30 days)
july_records = []
start_july = datetime(2024, 7, 1)

for day in range(30):
    timestamp = start_july + timedelta(days=day)
    
    for station in stations:
        # Warmer in July
        base_temp = 17 - (station["lat"] - 58) * 0.3
        temp = base_temp + np.random.normal(0, 1.5)
        
        july_records.append({
            "station_id": station["id"],
            "station_name": station["name"],
            "latitude": station["lat"],
            "longitude": station["lon"],
            "timestamp": timestamp.isoformat(),
            "depth_m": np.random.choice([5, 10, 25, 50, 100]),
            "temperature_c": round(temp, 2),
            "salinity_psu": round(34.5 + np.random.normal(0, 0.5), 2),
            "dissolved_oxygen_ml_l": round(6.5 + np.random.normal(0, 0.8), 2),
            "chlorophyll_ug_l": round(max(0.1, 3.0 + np.random.normal(0, 1.2)), 2)  # Higher in summer
        })

df_july = pd.DataFrame(july_records)
print(f"Generated {len(df_july)} July observations")
print(f"Temperature range: {df_july['temperature_c'].min():.1f}°C to {df_july['temperature_c'].max():.1f}°C")

In [ ]:
# Upload July data
csv_buffer2 = io.BytesIO()
df_july.to_csv(csv_buffer2, index=False)

file_id_july = dataset.files.upload("norwegian_sea_monitoring_july_2024.csv", csv_buffer2.getvalue())
print(f"Uploaded July data: {file_id_july}")

In [ ]:
# Append to existing table
print("Appending July data to table...")

dataset.files.ingest(file_id_july, opt="append")

# Verify
stats = dataset.table.stats()
print(f"\nTable now has {stats.num_rows:,} rows")
print(f"Expected: {len(df_stations) + len(df_july)} rows")

In [ ]:
# Query to verify date range
df_all = dataset.table.select(
    cols=["timestamp", "station_name", "temperature_c"]
).all().dataframe()

print(f"Full date range: {df_all['timestamp'].min()} to {df_all['timestamp'].max()}")
print(f"\nMonthly temperature averages:")
df_all['month'] = pd.to_datetime(df_all['timestamp']).dt.month_name()
print(df_all.groupby('month')['temperature_c'].mean())

## 9. Download Files

In [ ]:
# List all files
files = dataset.files.list()
print(f"Files in dataset ({len(files)}):")
for f in files:
    print(f"  {f.get('name', 'unnamed')} - {f.get('size', 0):,} bytes")

In [ ]:
# Download and verify
downloaded = b''
for chunk in dataset.files.download(file_id):
    downloaded += chunk

print(f"Downloaded {len(downloaded):,} bytes")

# Parse back to DataFrame
df_downloaded = pd.read_csv(io.BytesIO(downloaded))
print(f"Contains {len(df_downloaded)} rows")

## 10. Cleanup (Optional)

Remove test files if desired.

In [ ]:
# Optional: Clean up files
cleanup = input("Delete uploaded files? (yes/no): ").strip().lower()

if cleanup == 'yes':
    for f in dataset.files.list():
        fid = f.get('id')
        fname = f.get('name', fid)
        dataset.files.delete(fid)
        print(f"Deleted: {fname}")
    
    print(f"\nRemaining files: {len(dataset.files.list())}")
else:
    print("Files preserved for further exploration")

## Summary

This notebook demonstrated a complete data pipeline:

1. **Generate**: Create realistic oceanographic monitoring data
2. **Upload**: Send CSV to ODP dataset
3. **Metadata**: Add geographic extent and MIME type
4. **Ingest**: Convert to queryable table (`drop` mode)
5. **Query**: Filter and aggregate the data
6. **Append**: Add more data incrementally
7. **Download**: Retrieve files from ODP

## Ingest Mode Reference

| Mode | Behavior | Use Case |
|------|----------|----------|
| `drop` | Delete table, recreate from file | First load, schema changes |
| `truncate` | Clear rows, keep schema | Replace all data |
| `append` | Add rows to existing | Incremental updates |

## Next Steps

- **04_multi_dataset_join.ipynb**: Combine multiple datasets

## Resources

- [ODP Python SDK - Files](https://docs.hubocean.earth/python_sdk/intro/#files)
- [ODP My Data](https://app.hubocean.earth/) - Create your own datasets